In [1]:
import pandas as pd
from preprocess import Audio_Processor
import data_utils as du
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier

Using TensorFlow backend.


In [2]:
import warnings
warnings.filterwarnings('ignore')

# Setup

In [3]:
path_to_db='../ESC-50/'
ps = Audio_Processor(path_to_db + 'audio/')

# Load Dataset
Here we load the csv that describes each file in the dataset.

In [4]:
dataset = pd.read_csv(path_to_db + 'meta/esc50.csv')
classes = [None] * 50

In [5]:
for index, row in dataset.iterrows():
    target = row['target']
    classes[target] = row['category']

In [11]:
c_data = pd.concat([ps.process_fold(fld) for fld in range(1,6)], ignore_index=True)

In [12]:
n_categories = c_data.target.unique().size

In [13]:
c_data, scalar = du.normalize_data(c_data, 'target')
train, test = du.split_training_test(c_data, 0.2)
train_X = train.drop(columns=['target'])
train_y = train.target
test_X = test.drop(columns=['target'])
test_y = test.target

## Overall DNN
Train overall net for classifying all categories

In [15]:
clas = dnn_clas.create_baseline()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 38)                1482      
_________________________________________________________________
dropout_1 (Dropout)          (None, 38)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 38)                1482      
_________________________________________________________________
dropout_2 (Dropout)          (None, 38)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 50)                1950      
Total params: 4,914
Trainable params: 4,914
Non-trainable params: 0
_________________________________________________________________


In [16]:
dnn_clas.fit(clas, train_X, train_y, verbose=1)

Epoch 1/20
310994/310994 [==============================] - 25s 80us/step - loss: 3.1447 - acc: 0.1711
Epoch 2/20
310994/310994 [==============================] - 16s 51us/step - loss: 2.7351 - acc: 0.2584
Epoch 3/20
310994/310994 [==============================] - 16s 51us/step - loss: 2.6327 - acc: 0.2811
Epoch 4/20
310994/310994 [==============================] - 15s 49us/step - loss: 2.5870 - acc: 0.2909
Epoch 5/20
310994/310994 [==============================] - 15s 48us/step - loss: 2.5535 - acc: 0.2992
Epoch 6/20
310994/310994 [==============================] - 15s 48us/step - loss: 2.5377 - acc: 0.3024
Epoch 7/20
310994/310994 [==============================] - 15s 47us/step - loss: 2.5181 - acc: 0.3075
Epoch 8/20
310994/310994 [==============================] - 15s 48us/step - loss: 2.5094 - acc: 0.3102
Epoch 9/20
310994/310994 [==============================] - 15s 48us/step - loss: 2.5011 - acc: 0.3131
Epoch 10/20
310994/310994 [==============================] - 15s 49us/ste

In [17]:
dnn_clas.score(clas, test_X, test_y, verbose=1)

77749/77749 [==============================] - 119s 2ms/step
Test loss: 2.08641931449647
Test accuracy: 0.4369316647159449


## Scoring
Here we get the log likelihood of the categories when matched with its training data

In [ ]:
test = process_fold(5)
test, _ = du.normalize_data(test, 'target', scalar)
test_X = test.drop(columns=['target'])
test_y = test.target

In [ ]:
metrics.accuracy_score(test_y, list(clas.predict(test_X)))

In [ ]:
from classification_plots import plot_confusion_matrix
cm = metrics.confusion_matrix(test_y, list(clas.predict(test_X)))

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(25,25))
plot_confusion_matrix(cm, classes)

## Basic Probability of File Category
We run our classifier over the file's audio frames and use basic averages to determine the probability of it belonging to a class. (Should in future have classifier give probability scores for each category for each frame but... thats future work)

In [ ]:
print(classes)

In [ ]:
from collections import Counter
def file_probability(filename, clas, preprocessor, class_list):
    test_file = pd.DataFrame(preprocessor.preprocess(filename))
    predictions = [class_list[i] for i in list(clas.predict(test_file))]
    sns.countplot(predictions)
    c = Counter(predictions)
    return [(i, c[i] / len(predictions) * 100.0) for i, count in c.most_common()]

In [ ]:
plt.figure(figsize=(10,10))
file = dataset.filename.sample(n=1).values[0]
file_probability(file, clas, preprocess, classes)

In [ ]:
dataset[dataset.filename == file]